# IMPORT DEPENDENCIES

In [ ]:
import gym as gym
from gym import spaces
import pygame
import numpy as np
import random
from gym.error import DependencyNotInstalled
from typing import Optional, Union
import math
import numpy
import tensorflow as tf
import matplotlib.pyplot as plt
import os
from stable_baselines3 import PPO , DQN , A2C
from stable_baselines3.common.vec_env import VecFrameStack
from stable_baselines3.common.evaluation import evaluate_policy
import datetime
import torch as th


log_dir = "logs/Final/" 
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

# Env Settings

In [ ]:
TIMESTEP = 2000
NUMBER_OF_SHIPS=4   #max 8 ship
SCALAR =   2
MAP = 200 # bigger map train longer
Head_on_angle = 12.5
violation = 1
Arrival = 5
Collision = 5
TSP = 0.005
#shipdomain size = 10

# helper function 

In [ ]:
def distance(x1,y1,x2,y2):
    return math.sqrt(((x1-x2)**2) + ((y1-y2)**2))




import numpy as np

def calc_tcpa_dcpa(x1, y1, x2, y2, speed1, speed2, heading1, heading2):
    if distance(x1,y1,x2,y2) >= 10:
        return 0,0
    # convert headings to radians
    theta1 = np.deg2rad(heading1)
    theta2 = np.deg2rad(heading2)
    
    # calculate relative position and velocity vectors
    rel_pos = np.array([x2 - x1, y2 - y1])
    rel_vel = np.array([speed2 * np.cos(theta2), speed2 * np.sin(theta2)]) - np.array([speed1 * np.cos(theta1), speed1 * np.sin(theta1)])
    
    # calculate time to closest approach
    tcpa = -np.dot(rel_pos, rel_vel) / np.dot(rel_vel, rel_vel)
    if tcpa <= 0:
        return 0,0
           
    # calculate distance at TCPA
    dcpa = np.linalg.norm(rel_pos + tcpa * rel_vel)
    
    return tcpa, dcpa

def Headon(x1,y1,b1,x2,y2,b2):
    if distance(x1,y1,x2,y2) >= 10:
        return False
    else:
        relative_bearing = b1 - b2
        if relative_bearing <(180+Head_on_angle) and relative_bearing> (180-Head_on_angle):
            return True
        else:
            return False
        
def give_way_crossing(x1,y1,b1,x2,y2,b2):
    if distance(x1,y1,x2,y2) >= 10:
        pass
    else:
        relative_bearing = b1 - b2
        x_dist = x2-x1
        y_dist = y2-y1
        required = math.atan(y_dist/x_dist)*(180/np.pi)
        if abs(relative_bearing) > 180:
            return False
        if abs(relative_bearing) < 25:
            return False
        if x_dist >=0 and y_dist >= 0 :
            bearingr = required
        elif x_dist>=0 and y_dist <= 0 :
            bearingr = 90 + required
        elif x_dist<=0 and y_dist <=0:
            bearingr = 180 + required
        elif x_dist<=0 and y_dist>=0:
            bearingr = 360 - required
        if b1 - bearingr < 0:
            return True
        if b1 - bearingr >0:
            return False
        
        
def overtake_giveway(x1,y1,b1,x2,y2,b2):
    dist = distance(x1,y1,x2,y2)
    if  dist>= 10:
        return False
    else:
        relative_bearing = b1 - b2
        x_dist = x2-x1
        y_dist = y2-y1
        required = math.atan(y_dist/x_dist)*(180/np.pi)
        if relative_bearing >25 or relative_bearing < -25:
            return False
        if x_dist >=0 and y_dist >= 0 :
            bearingr = required
        elif x_dist>=0 and y_dist <= 0 :
            bearingr = 90 - required
        elif x_dist<=0 and y_dist <=0:
            bearingr = 180 - required
        elif x_dist<=0 and y_dist>=0:
            bearingr = abs(required)
        if abs(b1 - required >90):
            return False
        else:
            return True
        

def average_nonzero(arr):
    non_zero_values = arr[arr != 0]
    avg = np.mean(non_zero_values)
    return avg
        

# ENVIROMENT BUILDING

In [ ]:
class ssship(gym.Env):
    metadata = {
        "render_modes": ["human", "rgb_array"],
        "render_fps": 50,
    }
    def __init__(self, render_mode: Optional[str] = None):
        self.number_of_ships = NUMBER_OF_SHIPS
        self.window_size = 512
        self.action_space = spaces.Box(0,1,shape=(self.number_of_ships,),dtype=int)
        self.agents = [["ship_" + str(r) for r in range(self.number_of_ships)]]
        self.observation_space =  spaces.Box(0,MAP/SCALAR , shape=(self.number_of_ships,9), dtype=int)
        self.state =  np.zeros((self.number_of_ships,9))
        self.screen_width = MAP/SCALAR
        self.screen_height = self.screen_width
        self.render_mode = render_mode
        self.screen = None
        self.clock = None
        self.timestep = TIMESTEP
        self.reward = 0
        self.current = 0
        self.arrived = 0
        self.timetaken = 0
        
    def reset(self):
        
        self.state = np.zeros((self.number_of_ships,9))
        self.current = 0
        bad_random_destination = True
        bad_random_spawn = True
        for i in range(self.number_of_ships):
            x = random.randint(0,self.screen_width)
            y = random.randint(0,self.screen_width)
            s = random.randint(5,15)/10
            bearing = random.randint(0,360)
            Lx = random.randint(0,self.screen_width)
            Ly = random.randint(0,self.screen_width)
            le = random.randint(4,5)
            wi = random.randint(3,4)       
            while bad_random_destination:
                if distance(x,y,Lx,Ly) < 10:
                    Lx = random.randint(0,self.screen_width)
                    Ly = random.randint(0,self.screen_width)
                else:
                    bad_random_destination = False
            while bad_random_spawn:   
                if distance(self.state[i][0],self.state[i][1],self.state[i-1][0],self.state[i-1][1]) < 10 and i!=0:
                    x = random.randint(0,self.screen_width)
                    y = random.randint(0,self.screen_width)
                else:
                    bad_random_spawn = False
                    
                    
                
            Arrived = 0
            self.state[i] = [x,y,s,bearing,Lx,Ly,Arrived,le,wi]

        if self.render_mode == "human":
            self.render()
            
        self.timestep= TIMESTEP
        self.reward=0
        self.arrived=0
        
        
        
        return self.state

    def step(self,action):

        reward = 0
        self.timestep -=1
        reward -= TSP
        self.reward -= TSP

        
        
        for n in range(self.number_of_ships):
            bearing = self.state[n][3]
            
            
            ix = self.state[n][0]
            iy = self.state[n][1]
            x = self.state[n][0]
            y = self.state[n][1]
            if self.state[n][6] == 0 :

                if action[n]==0:
                    self.state[n][3] += 15
                if action[n]==1:
                    self.state[n][3] -= 15
                if action[n]==2:
                    self.state[n][2] += 0.1
                if action[n]==3:
                    self.state[n][2] -= 0.1
                if action[n]==4:
                    pass

                s = self.state[n][2]
                bearing = self.state[n][3]
                if (x<=0 or x>=self.screen_width) and (y<=0 and y>=self.screen_width):
                    pass

                elif (x>=self.screen_width) and (0<y<self.screen_width):
                    self.state[n][0] = self.screen_width -1
                    self.state[n][1] += s*np.cos((np.pi*bearing)/180)
                elif (x<=0) and (0<y<self.screen_width):
                    self.state[n][0] = 1
                    self.state[n][1] += s*np.cos((np.pi*bearing)/180)

                elif (y<=0) and (0<x<self.screen_width):
                    self.state[n][1] = 1
                    self.state[n][0] += s*np.sin((np.pi*bearing)/180)
                elif (y>=self.screen_width) and (0<x<self.screen_width):
                    self.state[n][1] = self.screen_width - 1
                    self.state[n][0] += s*np.sin((np.pi*bearing)/180)
                else:
                    self.state[n][0] += s*np.cos((np.pi*bearing)/180)
                    self.state[n][1] += s*np.sin((np.pi*bearing)/180)           
        

        terminated = False
        
        n_arrived = 0
        for i in range (self.number_of_ships):
            x = self.state[i][0]
            y = self.state[i][1]
            Lx = self.state[i][4]
            Ly = self.state[i][5]
            Arrived = self.state[i][6]
            
            e_distance = distance(x,y,Lx,Ly)
            pre_distance = distance(ix,iy,Lx,Ly)
            if e_distance <= 10/SCALAR and Arrived==0:
                reward += Arrival
                self.reward += Arrival
                self.state[i][6] = 1
                self.arrived +=1
            
                       
            if self.arrived == self.number_of_ships:
                terminated = True
                
  

        for i in range(self.number_of_ships):
            for j in range(self.number_of_ships):
                violate = False
                if i == j:
                    pass
                else:
                    x = self.state[i][0]
                    y = self.state[i][1]
                    x2 = self.state[j][0]
                    y2 = self.state[j][1]
                    b1 = self.state[i][3]
                    b2=self.state[j][3]
                    speed1= self.state[i][2]
                    speed2 = self.state[j][2]
                    distance_bw = distance(x,y,x2,y2)
                    
                    if self.state[j][6]==0 and self.state[i][6]==0:
                        tcpa,dcpa = calc_tcpa_dcpa(x, y, x2, y2, speed1, speed2, b1, b2)
                        reward -= 1 - math.exp(-(tcpa+dcpa))
                    
                    if distance_bw < 7.5/SCALAR and self.state[j][6]==0 and self.state[i][6]==0 :
                        reward -= Collision
                        self.reward-= Collision
                        terminated = True
                        

                    if self.state[j][6]==0 and self.state[i][6]==0:
                        gw = overtake_giveway(x,y,b1,x2,y2,b2)
                        gwc = give_way_crossing(x,y,b1,x2,y2,b2)
                        ho = Headon(x,y,b1,x2,y2,b2)

                        if gw and action[n] == 4:
                            violate == True

                        if gw and action[n] == 2:
                            violate == True
    #is slowing down but dont change direction considered give way?                        
                        if gwc and action[n] ==4:
                            violate == True

                        if gwc and action[n] ==2:
                            violate == True

                        if ho and action[n] == 1:
                            violate == True
                        if violate:
                            reward -= violation
        
            
        if self.timestep == 0:
            
            terminated = True
        
        if self.arrived == self.number_of_ships:
            terminated = True

            
            
            
        if self.render_mode == "human":
            self.render()

        info = {}

        return self.state , reward , terminated , info    

    def render(self):
        if self.render_mode is None:
            gym.logger.warn(
                "You are calling render method without specifying any render mode. "
                "You can specify the render_mode at initialization, "
                f'e.g. gym("{self.spec.id}", render_mode="rgb_array")'
            )
            return

        try:
            import pygame
            from pygame import gfxdraw
        except ImportError:
            raise DependencyNotInstalled(
                "pygame is not installed, run ``"
            )

        if self.screen is None:
            pygame.init()
            if self.render_mode == "human":
                pygame.display.init()
                self.screen = pygame.display.set_mode(
                    (self.screen_width* SCALAR, self.screen_height* SCALAR)
                )
            else:  # mode == "rgb_array":
                self.screen = pygame.Surface((self.screen_width* SCALAR, self.screen_height* SCALAR))

            if self.clock is None:
                self.clock = pygame.time.Clock()

        world_width = self.screen_width* SCALAR
        scale = self.screen_width* SCALAR / world_width* SCALAR

        self.surf = pygame.Surface((self.screen_width* SCALAR, self.screen_height* SCALAR))
        self.surf.fill((255, 255, 255))
        
        # color rendering
        black = (0,0,0)
        blue = (0,0,255)
        red = (255,0,0)
        green = (0,255,0)
        yellow = (127,255,0)
        pink = (255,20,147)
        gray=(128,128,128)
        turquoise = (0,206,209)
        colors = [black,blue,red,green,yellow,pink,gray,turquoise]
        
        
        for i in range(self.number_of_ships):
            x = self.state[i][0] * SCALAR
            y = self.state[i][1] * SCALAR
            width = 5 * SCALAR
            length = 2.5 * SCALAR
            bearing = self.state[i][3] 
            Lx = self.state[i][4] * SCALAR
            Ly = self.state[i][5] * SCALAR
            
            l,r,t,b = -width/2, width / 2, length / 2, -length / 2
            coords = [(l, b), (l, t), (r, t), (r, b)]
            coords = [(l, b), (l, t), (r, t), (r, b)]
            coords = [((c[0] )*np.cos(np.pi/180*bearing) - (c[1])*np.sin(np.pi/180*bearing) +x , (c[1])*np.cos(np.pi/180*bearing) + (c[0])*np.sin(np.pi/180*bearing)+y) for c in coords]
            
            gfxdraw.aapolygon(self.surf, coords, colors[i])
            gfxdraw.filled_polygon(self.surf, coords, colors[i])
            
            gfxdraw.circle(self.surf,int(Lx),int(Ly),10,colors[i])

        self.surf = pygame.transform.flip(self.surf, False, True)
        self.screen.blit(self.surf, (0, 0))
        my_font = pygame.font.SysFont('Comic Sans MS', 10)
        text_surface1 = my_font.render("reward:"+str((round(self.reward,2))), False, (0, 0, 0))
        self.screen.blit(text_surface1, (10,40))
        if self.render_mode == "human":
            pygame.event.pump()
            self.clock.tick(self.metadata["render_fps"])
            pygame.display.flip()

        elif self.render_mode == "rgb_array":
            return np.transpose(
                np.array(pygame.surfarray.pixels3d(self.screen)), axes=(1, 0, 2)
            )
        
    def close(self):
        if self.screen is not None:
            import pygame
            
            pygame.display.quit()
            pygame.quit()

            self.isopen = False
            

# ENV TESTING ( RANDOM TEST)

In [ ]:
env = ssship()

episodes = 10
scorelist = np.zeros((1,episodes))

for i in range (1):
    for episode in range(1, episodes+1):
        state = env.reset()
        done = False
        score = 0 
        collisions = 0

        while not done:
            #env.render()
            action = env.action_space.sample()
            n_state, reward, done, info = env.step(action)
            score+=reward
            
            
        scorelist[i][episode-1] = score
        #print('Episode:{} Score:{} '.format(episode, score))
    
env.close()
scorelist

In [ ]:
print(np.average(scorelist))

In [ ]:
env = ssship()


# Training PPO

In [ ]:
ppo_path = os.path.join('Training', 'PPO Saved Models', '1M_2S_C5_R10_ent0.05_PPO')

model = PPO("MlpPolicy", env, verbose=1, tensorboard_log=log_dir,clip_range=0.2,batch_size=64,ent_coef=0.05)
model.learn(total_timesteps=3_000_000,tb_log_name="3M_2S_C5_R10_PPO")
model.save(ppo_path)

# Training DQN


In [ ]:


class DQNAgent:
    def __init__(self, state_shape, action_shape, gamma=0.95, epsilon=1.0, epsilon_min=0.01, epsilon_decay=0.995, learning_rate=0.001, memory_size=100000):
        self.state_shape = state_shape
        self.action_shape = action_shape
        self.gamma = gamma
        self.epsilon = epsilon
        self.epsilon_min = epsilon_min
        self.epsilon_decay = epsilon_decay
        self.learning_rate = learning_rate
        self.memory = deque(maxlen=memory_size)
        self.model = self._build_model()
        self.target_model = self._build_model()
        self.update_target_model()

    def _build_model(self):
        model = tf.keras.models.Sequential([
            Dense(64, input_dim=self.state_shape[1], activation='relu'),
            Dense(64, activation='relu'),
            Dense(self.action_shape[1], activation='linear')
        ])
        model.compile(loss='mse', optimizer=Adam(lr=self.learning_rate))
        return model

    def update_target_model(self):
        self.target_model.set_weights(self.model.get_weights())

        
    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action.reshape(self.action_shape), reward, next_state, done))

    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return np.random.randint(low=0, high=2, size=self.action_shape)
        act_values = self.model.predict(state)
        return np.round(act_values[0]).astype(int)

    def replay(self, batch_size=32):
        if len(self.memory) < batch_size:
            return
        minibatch = random.sample(self.memory, batch_size)
        states, targets = [], []
        for state, action, reward, next_state, done in minibatch:
            target = self.model.predict(state)
            if done:
                target[0][np.argmax(action)] = reward
            else:
                t = self.target_model.predict(next_state)[0]
                target[0][np.argmax(action)] = reward + self.gamma * np.amax(t)
            states.append(state[0])
            targets.append(target[0])
        self.model.fit(np.array(states), np.array(targets), epochs=1, verbose=0)
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

    def load_model(self, name):
        self.model.load_weights(name)

    def save_model(self, name):
        self.model.save_weights(name)


state_shape = env.observation_space.shape
action_shape = env.action_space.shape
number_of_ships = NUMBER_OF_SHIPS
state_shape = (number_of_ships, 9)
action_shape = (number_of_ships, 1)
agent = DQNAgent(state_shape=state_shape, action_shape=action_shape)

episodes = 100
batch_size = 32
for episode in range(episodes):
    state = np.zeros((number_of_ships, 9))
    done = False
    score = 0
    while not done:
        action = agent.act(state)
        action = np.round(action).astype(int)
        next_state, reward, done, info = env.step(action)
        next_state = np.reshape(next_state, state_shape)
        reward = reward if not done else -10
        agent.remember(next_state, action, reward, next_state, done)
        state = next_state
        score += reward
        if done:
            agent.update_target_model()
            print("episode: {}/{}, score: {}".format(episode, episodes, score))
        agent.replay(batch_size)



# TRAINING A2C

In [ ]:
A2C_path = os.path.join('Training', 'A2C Saved Models', '1M_2S_C5_R10_A2C')
model = A2C("MlpPolicy", env, verbose=1, tensorboard_log=log_dir)
model.learn(total_timesteps=1_000_000,tb_log_name="1M_2S_C5_R10_A2C")
model.save(A2C_path)

# EVALUATE

In [ ]:
model.load(ppo_path)

In [ ]:
env = ssship(render_mode="human")
episodes = 5
obs = env.reset()
scorelist=[]
for episode in range(1, episodes+1):
    state = env.reset()
    dones = False
    score = 0 
    
    while not dones:
        #env.render()
        action, _states = model.predict(obs)
        obs, rewards, dones, info = env.step(action)
        
        score+=rewards
    scorelist.append(score)
    print('Episode:{} Score:{}'.format(episode, score))
print('average:' + str(np.average(scorelist)))
env.close()

# EVALUATE WITH COLLISION COUNT, ARRIVAL COUNT , TIME , VIOLATION

In [ ]:
class ssship(gym.Env):
    metadata = {
        "render_modes": ["human", "rgb_array"],
        "render_fps": 50,
    }
    def __init__(self, render_mode: Optional[str] = None):
        self.number_of_ships = NUMBER_OF_SHIPS
        self.window_size = 512
        self.action_space = spaces.Box(0,1,shape=(self.number_of_ships,),dtype=int)
        self.agents = [["ship_" + str(r) for r in range(self.number_of_ships)]]
        self.observation_space =  spaces.Box(0,MAP/SCALAR , shape=(self.number_of_ships,9), dtype=int)
        self.state =  np.zeros((self.number_of_ships,9))
        self.screen_width = MAP/SCALAR
        self.screen_height = self.screen_width
        self.render_mode = render_mode
        self.screen = None
        self.clock = None
        self.timestep = TIMESTEP
        self.reward = 0
        self.current = 0
        self.arrived = 0
        self.collided = 0
        self.violate = 0
        
    def reset(self):
        
        self.state = np.zeros((self.number_of_ships,9))
        self.current = 0
        bad_random_destination = True
        bad_random_spawn = True
        for i in range(self.number_of_ships):
            x = random.randint(0,self.screen_width)
            y = random.randint(0,self.screen_width)
            s = random.randint(5,15)/10
            bearing = random.randint(0,360)
            Lx = random.randint(0,self.screen_width)
            Ly = random.randint(0,self.screen_width)
            le = random.randint(4,5)
            wi = random.randint(3,4)       
            while bad_random_destination:
                if distance(x,y,Lx,Ly) < 10:
                    Lx = random.randint(0,self.screen_width)
                    Ly = random.randint(0,self.screen_width)
                else:
                    bad_random_destination = False
            while bad_random_spawn:   
                if distance(self.state[i][0],self.state[i][1],self.state[i-1][0],self.state[i-1][1]) < 10 and i!=0:
                    x = random.randint(0,self.screen_width)
                    y = random.randint(0,self.screen_width)
                else:
                    bad_random_spawn = False
                    
                    
                
            Arrived = 0
            self.state[i] = [x,y,s,bearing,Lx,Ly,Arrived,le,wi]

        if self.render_mode == "human":
            self.render()
            
        self.timestep= TIMESTEP
        self.reward=0
        self.arrived=0
        self.collided = 0
        self.timetaken = 0
        self.violate = 0
        
        return self.state

    def step(self,action):

        reward = 0
        self.timestep -=1
        reward -= TSP
        self.reward -= TSP
        self.arrived = 0
        self.timetaken = 1
        self.collided = 0
        self.violate = 0

        
        
        for n in range(self.number_of_ships):
            bearing = self.state[n][3]
            
            
            ix = self.state[n][0]
            iy = self.state[n][1]
            x = self.state[n][0]
            y = self.state[n][1]
            if self.state[n][6] == 0 :

                if action[n]==0:
                    self.state[n][3] += 15
                if action[n]==1:
                    self.state[n][3] -= 15
                if action[n]==2:
                    self.state[n][2] += 0.1
                if action[n]==3:
                    self.state[n][2] -= 0.1
                if action[n]==4:
                    pass

                s = self.state[n][2]
                bearing = self.state[n][3]
                if (x<=0 or x>=self.screen_width) and (y<=0 and y>=self.screen_width):
                    pass

                elif (x>=self.screen_width) and (0<y<self.screen_width):
                    self.state[n][0] = self.screen_width -1
                    self.state[n][1] += s*np.cos((np.pi*bearing)/180)
                elif (x<=0) and (0<y<self.screen_width):
                    self.state[n][0] = 1
                    self.state[n][1] += s*np.cos((np.pi*bearing)/180)

                elif (y<=0) and (0<x<self.screen_width):
                    self.state[n][1] = 1
                    self.state[n][0] += s*np.sin((np.pi*bearing)/180)
                elif (y>=self.screen_width) and (0<x<self.screen_width):
                    self.state[n][1] = self.screen_width - 1
                    self.state[n][0] += s*np.sin((np.pi*bearing)/180)
                else:
                    self.state[n][0] += s*np.cos((np.pi*bearing)/180)
                    self.state[n][1] += s*np.sin((np.pi*bearing)/180)           
        

        terminated = False
        
        n_arrived = 0
        for i in range (self.number_of_ships):
            x = self.state[i][0]
            y = self.state[i][1]
            Lx = self.state[i][4]
            Ly = self.state[i][5]
            Arrived = self.state[i][6]
            
            e_distance = distance(x,y,Lx,Ly)
            pre_distance = distance(ix,iy,Lx,Ly)
            if e_distance <= 10/SCALAR and Arrived==0:
                reward += Arrival
                self.reward += Arrival
                self.state[i][6] = 1
                self.arrived +=1
            
                       
            if self.arrived == self.number_of_ships:
                terminated = True
                   
        for i in range(self.number_of_ships):
            for j in range(self.number_of_ships):
                violate = False
                if i == j:
                    pass
                else:
                    x = self.state[i][0]
                    y = self.state[i][1]
                    x2 = self.state[j][0]
                    y2 = self.state[j][1]
                    b1 = self.state[i][3]
                    b2=self.state[j][3]
                    speed1= self.state[i][2]
                    speed2 = self.state[j][2]
                    distance_bw = distance(x,y,x2,y2)
                    
                    if self.state[j][6]==0 and self.state[i][6]==0:
                        tcpa,dcpa = calc_tcpa_dcpa(x, y, x2, y2, speed1, speed2, b1, b2)
                        reward -= 1 - math.exp(-(tcpa+dcpa))
                    
                    if distance_bw < 7.5/SCALAR and self.state[j][6]==0 and self.state[i][6]==0 :
                        reward -= Collision
                        self.reward-= Collision
                        terminated = True
                        

                    if self.state[j][6]==0 and self.state[i][6]==0:
                        gw = overtake_giveway(x,y,b1,x2,y2,b2)
                        gwc = give_way_crossing(x,y,b1,x2,y2,b2)
                        ho = Headon(x,y,b1,x2,y2,b2)

                        if gw and action[n] == 4:
                            violate == True

                        if gw and action[n] == 2:
                            violate == True
    #is slowing down but dont change direction considered give way?                        
                        if gwc and action[n] ==4:
                            violate == True

                        if gwc and action[n] ==2:
                            violate == True

                        if ho and action[n] == 1:
                            violate == True
                        if violate:
                            self.violate += 1
                            reward -= violation

            
        if self.timestep == 0:
            
            terminated = True
        TE = 0
        
        if self.arrived == self.number_of_ships:
            TE = self.timetaken
            terminated = True
                   
        if self.render_mode == "human":
            self.render()

        info = {}
        
        

        return self.state , reward , terminated , info  , self.arrived , self.collided , TE , self.violate

    def render(self):
        if self.render_mode is None:
            gym.logger.warn(
                "You are calling render method without specifying any render mode. "
                "You can specify the render_mode at initialization, "
                f'e.g. gym("{self.spec.id}", render_mode="rgb_array")'
            )
            return

        try:
            import pygame
            from pygame import gfxdraw
        except ImportError:
            raise DependencyNotInstalled(
                "pygame is not installed, run ``"
            )

        if self.screen is None:
            pygame.init()
            if self.render_mode == "human":
                pygame.display.init()
                self.screen = pygame.display.set_mode(
                    (self.screen_width* SCALAR, self.screen_height* SCALAR)
                )
            else:  # mode == "rgb_array":
                self.screen = pygame.Surface((self.screen_width* SCALAR, self.screen_height* SCALAR))

            if self.clock is None:
                self.clock = pygame.time.Clock()

        world_width = self.screen_width* SCALAR
        scale = self.screen_width* SCALAR / world_width* SCALAR

        self.surf = pygame.Surface((self.screen_width* SCALAR, self.screen_height* SCALAR))
        self.surf.fill((255, 255, 255))
        
        # color rendering
        black = (0,0,0)
        blue = (0,0,255)
        red = (255,0,0)
        green = (0,255,0)
        yellow = (127,255,0)
        pink = (255,20,147)
        gray=(128,128,128)
        turquoise = (0,206,209)
        colors = [black,blue,red,green,yellow,pink,gray,turquoise]
        
        
        for i in range(self.number_of_ships):
            x = self.state[i][0] * SCALAR
            y = self.state[i][1] * SCALAR
            width = 5 * SCALAR
            length = 2.5 * SCALAR
            bearing = self.state[i][3] 
            Lx = self.state[i][4] * SCALAR
            Ly = self.state[i][5] * SCALAR
            
            l,r,t,b = -width/2, width / 2, length / 2, -length / 2
            coords = [(l, b), (l, t), (r, t), (r, b)]
            coords = [(l, b), (l, t), (r, t), (r, b)]
            coords = [((c[0] )*np.cos(np.pi/180*bearing) - (c[1])*np.sin(np.pi/180*bearing) +x , (c[1])*np.cos(np.pi/180*bearing) + (c[0])*np.sin(np.pi/180*bearing)+y) for c in coords]
            
            gfxdraw.aapolygon(self.surf, coords, colors[i])
            gfxdraw.filled_polygon(self.surf, coords, colors[i])
            
            gfxdraw.circle(self.surf,int(Lx),int(Ly),10,colors[i])

        self.surf = pygame.transform.flip(self.surf, False, True)
        self.screen.blit(self.surf, (0, 0))
        my_font = pygame.font.SysFont('Comic Sans MS', 10)
        text_surface1 = my_font.render("reward:"+str((round(self.reward,2))), False, (0, 0, 0))
        self.screen.blit(text_surface1, (10,40))
        if self.render_mode == "human":
            pygame.event.pump()
            self.clock.tick(self.metadata["render_fps"])
            pygame.display.flip()

        elif self.render_mode == "rgb_array":
            return np.transpose(
                np.array(pygame.surfarray.pixels3d(self.screen)), axes=(1, 0, 2)
            )
        
    def close(self):
        if self.screen is not None:
            import pygame
            
            pygame.display.quit()
            pygame.quit()

            self.isopen = False
            

In [ ]:
env = ssship()

episodes = 20
scorelist = np.zeros((1,episodes))
arrlist = np.zeros((1,episodes))
collist = np.zeros((1,episodes))
timelist = np.zeros((1,episodes))
violist = np.zeros((1,episodes))


for i in range (1):
    for episode in range(1, episodes+1):
        state = env.reset()
        done = False
        score = 0 
        collisions = 0
        arrived= 0
        time = 0
        viol=0
        

        while not done:
            #env.render()
            action, _states = model.predict(obs)
            n_state, reward, done, info,arr,col,te,vio = env.step(action)
            score+=reward
            arrived += arr
            collisions += col
            time += te
            viol += vio
            
            

        scorelist[i][episode-1] = score
        arrlist[i][episode-1] = arrived
        collist[i][episode-1] = collisions
        timelist[i][episode-1] = time
        violist[i][episode-1] = viol
        
        #print('Episode:{} Score:{} '.format(episode, score))
    
env.close()
scorelist
print(np.average(scorelist))



print('number of episodes with collision: ' + str(np.count_nonzero(collist == 1)))
print('number of arrival,average per episode: ' + str(np.average(arrlist) ))
print('average time taken: ' + str(average_nonzero(timelist)))
print('average violation: ' + str(np.average(violist) ))

In [ ]:
env = ssship()

episodes = 20
scorelist = np.zeros((1,episodes))
arrlist = np.zeros((1,episodes))
collist = np.zeros((1,episodes))
timelist = np.zeros((1,episodes))
violist = np.zeros((1,episodes))

for i in range (1):
    for episode in range(1, episodes+1):
        state = env.reset()
        done = False
        score = 0 
        collisions = 0
        arrived= 0
        time = 0
        
        while not done:
            #env.render()
            action = env.action_space.sample()
            n_state, reward, done, info,arr,col,te,vio = env.step(action)
            score+=reward
            arrived += arr
            collisions += col
            time += te
            viol += vio
            
        scorelist[i][episode-1] = score
        arrlist[i][episode-1] = arrived
        collist[i][episode-1] = collisions
        timelist[i][episode-1] = time
        violist[i][episode-1] = viol
        
        #print('Episode:{} Score:{} '.format(episode, score))
    
env.close()
scorelist
print(np.average(scorelist))

print(collist)
print(timelist)

print('number of episodes with collision: ' + str(np.count_nonzero(collist == 1)))
print('number of arrival,average per episode: ' + str(np.average(arrlist) ))
print('average time taken: ' + str(average_nonzero(timelist)))
print('average violation: ' + str(np.average(violist) ))

In [ ]:

%load_ext tensorboard
import tensorflow as tf
import datetime


In [ ]:

%tensorboard --logdir {log_dir}  


In [ ]:
#To-Do 17/2

# can remove ship size if got time 

#reminder
# DQN training, train seperately , unistall and install the older version of tensorflow and change action shape. 
# After training , change back if want to train ALG